In [ ]:
import io
import torch
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
import seaborn as sns
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split
from functools import partial
from IPython.display import HTML
import math
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import torch
import torch.nn as nn
import sklearn
import torch.nn.functional as F
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Bucharest_HousePriceDataset.csv to Bucharest_HousePriceDataset (1).csv


Folosim functia upload de la google colab pentru a citi fisierul

In [ ]:
df1 = pd.read_csv(io.BytesIO(uploaded['Bucharest_HousePriceDataset.csv']), delimiter=',')
print(df1)

      Nr Camere  Suprafata  Etaj  Total Etaje  Sector  Scor    Pret
0             4     108.00     2            3       4     5   83000
1             1      41.00     1            8       1     1   39900
2             3      63.52     1            3       2     3   84900
3             1      33.00     3           10       5     1   45500
4             2      62.00     5            9       5     5   54900
...         ...        ...   ...          ...     ...   ...     ...
3524          3     102.00     2            5       1     1  189000
3525          4      96.00     1            3       3     2  200000
3526          1      35.00     1            3       4     5   47000
3527          2      60.00     6            7       1     2   89500
3528          3      80.00     3            5       1     2  139000

[3529 rows x 7 columns]


In [ ]:
class GDLinearRegression(nn.Module):
  def __init__(self):
    super().__init__()

    self.w = nn.Parameter(torch.randn(6, requires_grad = True))
    self.b = nn.Parameter(torch.randn(1, requires_grad = True))

  def __call__(self, x: torch.Tensor) -> torch.Tensor:
    x = torch.Tensor(x)
    result = x @ self.w+self.b
    return result

  def zero_grad(self):
    self.w.grad.zero_()
    self.b.grad.zero_()

In [ ]:
class MSE():
  
  def __call__(self, x: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    x = torch.Tensor(x)
    target = torch.Tensor(target)
    mse = ((x-target)**2).sum().sqrt().mean()
    return mse

In [ ]:
class GD():
  """Gradient Descent optimizer"""

  def __init__(self, params: torch.Tensor, lr: int):
    self.w, self.b = list(params)
    self.lr = lr


  def step(self):
    self.w -= self.lr*self.w.grad

    self.b -= self.lr*self.b.grad 


In [ ]:
def train(model: GDLinearRegression, data: torch.Tensor, 
          labels: torch.Tensor, optim: GD, criterion: MSE):

  predictions = model(data)
  loss = criterion(labels,predictions) 
  loss.backward() 

  
  with torch.no_grad():
    optim.step()
    model.zero_grad()
  
  return model

In [ ]:
X=df1.drop(columns='Nr Camere').values

y=df1[['Nr Camere']].values.ravel()

print(y)
x_train, x_valid, y_train, y_valid = train_test_split(X,y, train_size = 0.8)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_train = torch.tensor(x_train).float()

#Folosim aceeasi deviatie standard de la training pentru test
x_valid = std_scale.transform(x_valid)
x_valid = torch.tensor(x_valid).float()

mse = MSE()

[4 1 3 ... 1 2 3]


In [ ]:
best_loss=10**1000
best_lr = 0
total_steps = 100
for idx in np.linspace(0.001,20,100):
    lr = idx
    total_steps = 100

    model = GDLinearRegression()
    optimizer = GD(model.parameters(), lr=lr)
    criterion = MSE()
    
    for i in range(total_steps):
        train(model, x_train, y_train, optimizer, criterion)

    with torch.no_grad():
        y_pred = model(x_train)
    if best_loss > mse(y_pred,y_train).item():
        best_loss = mse(y_pred,y_train).item()
        best_lr = idx
    
print(best_lr)

0.001


In [ ]:
lr = best_lr
total_steps = 500
model = GDLinearRegression()
optimizer = GD(model.parameters(), lr=lr)
criterion = MSE()

for i in range(total_steps):
    train(model, x_train, y_train, optimizer, criterion)

with torch.no_grad():
    y_pred = model(x_train)

Best lr


In [ ]:
with torch.no_grad():
    y_pred = model(x_train)
predicted = torch.round(y_pred).numpy()
accuracy = (predicted==y_train).sum()/predicted.shape[0]
print(accuracy)

0.6875664187035069


Acc pe train

In [ ]:
with torch.no_grad():
    y_pred = model(x_valid)
predicted = torch.round(y_pred).numpy()
accuracy = (predicted==y_valid).sum()/predicted.shape[0]
print(accuracy)

0.6827195467422096


Acc pe val

In [ ]:
regression_matrix = confusion_matrix(predicted,y_valid)
regression_mse = mse(predicted,y_valid)
print(regression_matrix)


[[ 13   0   0   0   0   0   0]
 [ 56 308  64   0   0   0   0]
 [  0  24 145  48   2   0   0]
 [  0   0  17  14   5   0   0]
 [  0   0   1   1   1   0   0]
 [  0   0   0   2   1   1   1]
 [  0   0   0   1   1   0   0]]


In [ ]:
X=df1.drop(columns='Nr Camere').values

y=df1[['Nr Camere']].values
y=y-1
x_train, x_valid, y_train, y_valid = train_test_split(X,y, train_size = 0.8)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_train = torch.tensor(x_train).float()


x_valid = std_scale.transform(x_valid)
x_valid = torch.tensor(x_valid).float()

In [ ]:
class TwoLayer(nn.Module):
    def __init__(self, in_size: int, hidden_size: int, out_size: int):
        super().__init__()
        self._layer1 = nn.Linear(in_size, hidden_size)
        self._layer2 = nn.Linear(hidden_size, out_size)
    def forward(self, x):
        x = self._layer1(x)
        x = torch.relu(x)
    
        x = self._layer2(x)
        return x

In [ ]:
model = TwoLayer(6,5000,9)
NUM_EPOCHS = 2000

optim = torch.optim.SGD(model.parameters(), lr=2.5)
for i in range(NUM_EPOCHS):
    print(i)
    model.train()
    optim.zero_grad()
    output = model(x_train)
    criterion = nn.CrossEntropyLoss()
    target = torch.tensor(y_train).long().squeeze(1)
    print(target)
    loss = criterion(output, target)
    loss.backward()
    optim.step()

In [ ]:
predicted = np.array(torch.argmax(model(x_train), dim=-1))
accuracy = (predicted==y_train.ravel()).sum()/predicted.shape[0]
print(accuracy)

0.9036486007793127


In [ ]:
predicted = np.array(torch.argmax(model(x_valid), dim=-1))
accuracy = (predicted==y_valid.ravel()).sum()/predicted.shape[0]
print(accuracy)

0.7705382436260623


In [ ]:
class_matrix = confusion_matrix(predicted,y_valid)
class_mse = mse(predicted,y_valid)

print(class_matrix)

[[ 51   8   1   0   0   0   0]
 [ 23 270  42   2   0   0   0]
 [  0  22 184  17   0   0   0]
 [  0   2  30  37   5   0   0]
 [  0   0   5   1   2   2   2]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]]


In [ ]:
print('Clasificare Nr. Camere\n', class_matrix)
print('Regresie Nr. Camere\n',regression_matrix)

Clasificare Nr. Camere
 [[ 51   8   1   0   0   0   0]
 [ 23 270  42   2   0   0   0]
 [  0  22 184  17   0   0   0]
 [  0   2  30  37   5   0   0]
 [  0   0   5   1   2   2   2]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]]
Regresie Nr. Camere
 [[ 13   0   0   0   0   0   0]
 [ 56 308  64   0   0   0   0]
 [  0  24 145  48   2   0   0]
 [  0   0  17  14   5   0   0]
 [  0   0   1   1   1   0   0]
 [  0   0   0   2   1   1   1]
 [  0   0   0   1   1   0   0]]


In [ ]:
print('Clasificare scor\n', class_mse)
print('Regresie Nr. Camere\n',regression_mse)

Clasificare scor
 tensor(873.5365)
Regresie Nr. Camere
 tensor(15.8114)
